In [1]:
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
import psycopg2

In [2]:
database_name = "deneme"
user_name = "postgres"
password = "****"
host_ip = "127.0.0.1"
host_port = "5432"

baglanti = psycopg2.connect(database=database_name,
                            user=user_name,
                            password=password,
                            host=host_ip,
                            port=host_port)
baglanti.autocommit = True
cursor = baglanti.cursor()

In [3]:
query_create_table = """
CREATE TABLE IF NOT EXISTS terorListeleri(
id SERIAL PRIMARY KEY,
isim TEXT NOT NULL,
soyisim TEXT NOT NULL,
dogumyeri TEXT,
dogumtarihi TEXT,
bulunduguorgut TEXT,
renkkategori TEXT,
resimurl TEXT
)
"""
cursor.execute(query_create_table)

In [4]:
browser = webdriver.Chrome("C:/Users/****/Desktop/chromedriver")
browser.get("http://www.terorarananlar.pol.tr/tarananlar")
time.sleep(1)
browser.fullscreen_window()
time.sleep(1)

C:\Users\****\AppData\Local\Temp\ipykernel_5944\3597576304.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome("C:/Users/****/Desktop/chromedriver")


In [5]:
def dahafazlagoster():
    daha_fazla_goster = browser.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div/div[2]/div/div[2]/div/div/div/div/div/div/div[3]/div[2]/a")
    try:
        for i in range(1,70):
            daha_fazla_goster.click()
    except:
        pass


In [6]:
def renkbolumu(liste_adi):
    if liste_adi == "kirmiziliste.txt":
        renkbutonu = "button[1]"
        init = 1
        ikucuk = 9
        ibuyuk = 50
        kategoriadi = "Kırmızı"
    elif liste_adi == "maviliste.txt":
        renkbutonu = "button[2]"
        init = 11
        ikucuk = 20
        ibuyuk = 321
        kategoriadi = "Mavi"
    elif liste_adi == "yesilliste.txt":
        renkbutonu = "button[3]"
        init = 31
        ikucuk = 40
        ibuyuk = 417
        kategoriadi = "Yeşil"
    elif liste_adi == "turunculiste.txt":
        renkbutonu = "button[4]"
        init = 21
        ikucuk = 30
        ibuyuk = 550
        kategoriadi = "Turuncu"
    elif liste_adi == "griliste.txt":
        renkbutonu = "button[5]"
        init = 41
        ikucuk = 50
        ibuyuk = 787
        kategoriadi = "Gri"
    
    arananlar_list = browser.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div/div[2]/div/div[2]/div/div/div/div/div/div/div[1]/"+renkbutonu)
    arananlar_list.click()
    dahafazlagoster()

    bilgilistedekiler = browser.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div/div[2]/div/div[2]/div/div/div/div/div/div/div[3]")
    urlkisiler = []
    ilkdeger = init
    for i in range(init,1555):
        if i > ikucuk and i < ibuyuk:
            ilkdeger += 1
            continue
        else:

            ilkblok = "'/html/body/div[1]/div/div[2]/div/div/div[2]/div/div[2]/div/div/div/div/div/div/div[3]/div[1]/div["
            sonblok = "]/div[1]'"
            str_i = str(ilkdeger)
            kisiurl = ilkblok+str_i+sonblok
            kisiurl = kisiurl[1:-1]
            ilkdeger = int(ilkdeger)
                
        urlkisiler.append(kisiurl)
        
        ilkdeger += 1
    
    
    resimler = []
    try:

        for i in urlkisiler:
            urlkisi = browser.find_element_by_xpath(i).value_of_css_property('background-image')
            urlkisi = urlkisi[5:-2]
            resimler.append(urlkisi)
            

    except:
        pass
    listler = []
    for kisi in bilgilistedekiler.text:
        listler.append(bilgilistedekiler.text)
        break

    with open (liste_adi, "w",encoding="UTF-8") as file:
            for kisi in listler:
                file.write(kisi + "\n")

    listler2 = []
    with open (liste_adi, "r",encoding="UTf-8") as file:
            for item in file:
                listler2.append(item)

    
    terorlistesitamami = []
    i = 0
    u = 0
    listeuzunlugu = len(listler2)
    while i < listeuzunlugu:
        terorlistesi = ()
        ilkblok = listler2[i:i+3]
        isimblok = ilkblok[0]
        isimbloksplit = isimblok.split()
        soyad = isimbloksplit[-1]
        isimbloksplitad = isimbloksplit[:-1]
        isimsiz = ''
        orgut = ilkblok[-1]
        for a in isimbloksplitad:
            isimsiz = isimsiz + ' ' + a 
        sehirtarih = listler2[i+1]
        sehirtarihayir = sehirtarih.split()
        sehir = sehirtarihayir[0]
        tarih = sehirtarihayir[2]
        if tarih.isdigit():
            pass
        else:
            tarih = sehirtarihayir[-1]
            sehir = sehirtarihayir[0] + ' ' + sehirtarihayir[1]

        if len(orgut.split()) <= 3:
                
            orgutayir = orgut.split()
            orgutunadi = orgutayir[:-2]
        else:
            orgut2 = orgut.split()
            orgutunadi[0] = orgut2[0] + ' ' + orgut2[1]
        
        
        try:
            terorlistesi = (isimsiz.strip(),soyad,sehir,tarih,orgutunadi[0],kategoriadi,resimler[u])
            terorlistesitamami.append(tuple(terorlistesi))
        except:
            if u > 1000:
                u += 1
                continue
            else:
                pass
            
        i += 3
        u += 1
    terorListeleri_record = ", ".join(["%s"] * len(terorlistesitamami))
    query_insert = (
    f"INSERT INTO terorListeleri(isim, soyisim, dogumyeri, dogumtarihi, bulunduguorgut, renkkategori, resimurl) VALUES {terorListeleri_record}"
    )
    cursor.execute(query_insert, terorlistesitamami)
    
  
    

In [7]:
browser.execute_script('window.scrollTo(0,document.body.scrollTop)')
renkbolumu("kirmiziliste.txt")

C:\Users\****\AppData\Local\Temp\ipykernel_5944\2871815445.py:33: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  arananlar_list = browser.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div/div[2]/div/div[2]/div/div/div/div/div/div/div[1]/"+renkbutonu)
C:\Users\****\AppData\Local\Temp\ipykernel_5944\188772635.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  daha_fazla_goster = browser.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div/div[2]/div/div[2]/div/div/div/div/div/div/div[3]/div[2]/a")
C:\Users\****\AppData\Local\Temp\ipykernel_5944\2871815445.py:37: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  bilgilistedekiler = browser.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div/div[2]/div/div[2]/div/div/div/div/div/div/div[3]")
C:\Users\****\AppD

In [8]:
browser.execute_script('window.scrollTo(0,document.body.scrollTop)')
renkbolumu("maviliste.txt")

C:\Users\****\AppData\Local\Temp\ipykernel_5944\2871815445.py:33: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  arananlar_list = browser.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div/div[2]/div/div[2]/div/div/div/div/div/div/div[1]/"+renkbutonu)
C:\Users\****\AppData\Local\Temp\ipykernel_5944\188772635.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  daha_fazla_goster = browser.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div/div[2]/div/div[2]/div/div/div/div/div/div/div[3]/div[2]/a")
C:\Users\****\AppData\Local\Temp\ipykernel_5944\2871815445.py:37: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  bilgilistedekiler = browser.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div/div[2]/div/div[2]/div/div/div/div/div/div/div[3]")
C:\Users\****\AppD

In [9]:
browser.execute_script('window.scrollTo(0,document.body.scrollTop)')
renkbolumu("yesilliste.txt")

C:\Users\****\AppData\Local\Temp\ipykernel_5944\2871815445.py:33: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  arananlar_list = browser.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div/div[2]/div/div[2]/div/div/div/div/div/div/div[1]/"+renkbutonu)
C:\Users\****\AppData\Local\Temp\ipykernel_5944\188772635.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  daha_fazla_goster = browser.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div/div[2]/div/div[2]/div/div/div/div/div/div/div[3]/div[2]/a")
C:\Users\****\AppData\Local\Temp\ipykernel_5944\2871815445.py:37: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  bilgilistedekiler = browser.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div/div[2]/div/div[2]/div/div/div/div/div/div/div[3]")
C:\Users\****\AppD

In [10]:
browser.execute_script('window.scrollTo(0,document.body.scrollTop)')
renkbolumu("turunculiste.txt")

C:\Users\****\AppData\Local\Temp\ipykernel_5944\2871815445.py:33: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  arananlar_list = browser.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div/div[2]/div/div[2]/div/div/div/div/div/div/div[1]/"+renkbutonu)
C:\Users\****\AppData\Local\Temp\ipykernel_5944\188772635.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  daha_fazla_goster = browser.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div/div[2]/div/div[2]/div/div/div/div/div/div/div[3]/div[2]/a")
C:\Users\****\AppData\Local\Temp\ipykernel_5944\2871815445.py:37: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  bilgilistedekiler = browser.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div/div[2]/div/div[2]/div/div/div/div/div/div/div[3]")
C:\Users\****\AppD

In [11]:
browser.execute_script('window.scrollTo(0,document.body.scrollTop)')
renkbolumu("griliste.txt")

C:\Users\****\AppData\Local\Temp\ipykernel_5944\2871815445.py:33: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  arananlar_list = browser.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div/div[2]/div/div[2]/div/div/div/div/div/div/div[1]/"+renkbutonu)
C:\Users\****\AppData\Local\Temp\ipykernel_5944\188772635.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  daha_fazla_goster = browser.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div/div[2]/div/div[2]/div/div/div/div/div/div/div[3]/div[2]/a")
C:\Users\****\AppData\Local\Temp\ipykernel_5944\2871815445.py:37: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  bilgilistedekiler = browser.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div/div[2]/div/div[2]/div/div/div/div/div/div/div[3]")
C:\Users\****\AppD

In [12]:
browser.close()